In [1]:
!pip3 install crypto


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 kB 5.2 MB/s eta 0:00:00


In [2]:
!pip3 install pycrypto

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 446.2/446.2 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pycrypto: filename=pycrypto-2.6.1-cp310-cp310-linux_x86_64.whl size=498546 sha256=6e96ec71b6b1c2b060c9b926e38c01d09f9ee55d8360e7fc6702dd26d1311997
  Stored in directory: /root/.cache/pip/wheels/e8/4b/5b/b10a6fc885057b6ff9fbd5691d7e700d0a9408f80b7e6f12e0
Successfully built pycrypto


In [3]:
!pip3 install pycryptodome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.2 MB/s eta 0:00:00


In [4]:
import hashlib
import random
import binascii
import datetime
import collections
from Crypto.PublicKey import RSA
from Crypto import Random
from Crypto.Cipher import PKCS1_v1_5
from collections import OrderedDict
import Crypto
import Crypto.Random
from Crypto.Hash import SHA
from Crypto.Signature import PKCS1_v1_5

class Client:
    def __init__(self):
        rand_func = Random.new().read
        self._private_key = RSA.generate(1024, rand_func)
        self._public_key = self._private_key.publickey()
        self._signer = PKCS1_v1_5.new(self._private_key)

    @property
    def identity(self):
        return binascii.hexlify(self._public_key.exportKey(format='DER')).decode('ascii')

class Transaction:
    def __init__(self, sender, recipient, value):
        self.sender = sender
        self.recipient = recipient
        self.value = value
        self.time = datetime.datetime.now()

    def to_dict(self):
        if self.sender == "Genesis":
            identity = "Genesis"
        else:
            identity = self.sender.identity
        return collections.OrderedDict({
            'sender': identity,
            'recipient': self.recipient,
            'value': self.value,
            'time' : self.time})

    def sign_transaction(self):
        private_key = self.sender._private_key
        signer = PKCS1_v1_5.new(private_key)
        h = SHA.new(str(self.to_dict()).encode('utf8'))
        return binascii.hexlify(signer.sign(h)).decode('ascii')

def display_transaction(transaction):
    dict = transaction.to_dict()
    print ("sender: " + dict['sender'])
    print ('----- ')
    print ("recipient: " + dict['recipient'])
    print ('----- ')
    print ("value: " + str(dict['value']))
    print ('----- ')
    print ("time: " + str(dict['time']))
    print ('----- ')

def dump_blockchain(blockchain):
    print ("Number of blocks in the chain: " + str(len(blockchain)))
    for x in range(len(blockchain)):
        block_temp = blockchain[x]
        print ("block # " + str(x))
        for transaction in block_temp.verified_transactions:
            display_transaction(transaction)
        print ('---------------')
    print ('=====================================')

class Block:
    def __init__(self):
        self.verified_transactions = []
        self.previous_block_hash = ""
        self.Nonce = ""

def sha256(message):
    return hashlib.sha256(message.encode('ascii')).hexdigest()

def mine(block, difficulty=1):
    assert difficulty >= 1
    prefix = '0' * difficulty
    for i in range(1000):
        digest = sha256(str(block) + str(i))
        if digest.startswith(prefix):
            return i  # i= nonce value

A = Client()
B = Client()
C = Client()

t0 = Transaction("Genesis", A.identity, 500.0)
t1 = Transaction(A, B.identity, 40.0)
t2 = Transaction(A, C.identity, 70.0)
t3 = Transaction(B, C.identity, 700.0)

TPCoins = []

block0 = Block()
block0.previous_block_hash = None
block0.verified_transactions.append(t0)
TPCoins.append(block0)

block1 = Block()
block1.previous_block_hash = hashlib.sha256(str(block0).encode()).hexdigest()
block1.verified_transactions.append(t1)
block1.verified_transactions.append(t2)
block1.Nonce = mine(block1, 2)
TPCoins.append(block1)

block2 = Block()
block2.previous_block_hash = hashlib.sha256(str(block1).encode()).hexdigest()
block2.verified_transactions.append(t3)
block2.Nonce = mine(block2, 2)
TPCoins.append(block2)

dump_blockchain(TPCoins)


Number of blocks in the chain: 3
block # 0
sender: Genesis
----- 
recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100936d6302f52e87471756e4a649fa012d894736bf038478f6255d2425da742fefe99f391803837555d32e6d3713df458e1bb3847d8643b94f6e66a9b0241a0bb823b3581d3926bfedb0c5fa8bf0bf732595f689b9809b220522d94dbd3da1911fd226692b14848bdd8a556b9182e9543f5537cb87d772b853739155f7299a2f4b0203010001
----- 
value: 500.0
----- 
time: 2024-04-16 09:21:20.874580
----- 
---------------
block # 1
sender: 30819f300d06092a864886f70d010101050003818d0030818902818100936d6302f52e87471756e4a649fa012d894736bf038478f6255d2425da742fefe99f391803837555d32e6d3713df458e1bb3847d8643b94f6e66a9b0241a0bb823b3581d3926bfedb0c5fa8bf0bf732595f689b9809b220522d94dbd3da1911fd226692b14848bdd8a556b9182e9543f5537cb87d772b853739155f7299a2f4b0203010001
----- 
recipient: 30819f300d06092a864886f70d010101050003818d0030818902818100a3cbd480a8b187b13ef2d8136db66486ad8ef0c6a039dcea1fe2339430866793b0f4ca671ab0b8057bb13a052eb4dce9